In [38]:
import numpy as np
import pandas as pd
import pickle 
import torch
from model import BERT4Rec

In [39]:
if torch.cuda.is_available() :
    cuda_aval = True
device = torch.device("cuda" if cuda_aval else "cpu")

In [40]:
with open("./data/input_data.pickle", "rb") as fr :
    raw_data = pickle.load(fr)

unique_sid = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))

item_id = unique_sid.to_numpy().reshape(6807)
item_id = np.insert(item_id, 0, 0)
uim = pd.read_csv("/opt/ml/input/workspace/CF/DL CF/Mult-VAE/new_user_movie_interaction.csv")
user_id = uim["user"].to_numpy()

In [41]:
model = torch.load("./exp/model.pt").to(device)
#model = BERT4Rec()
#model.load_state_dict(torch.load("./exp/model.pth", map_location = device))

In [58]:
max_len = 100
final = list()
cnt = 0
for key in raw_data.keys() :

    length = len(raw_data[key])
    if length < max_len : 
        dif = max_len-length
        data = [0]*dif + raw_data[key][-length:]
    else :
        data = raw_data[key][-max_len:]

    data = torch.LongTensor(data).unsqueeze(dim=0)
    result = model(data)[0]
    #t = result.detach().cpu().sum(axis=0)
    t = result[0].detach().cpu()
    t[data] = -np.inf
    top_k_idx = np.argpartition(t, -10)[-10:]
    rec_item_id = item_id[top_k_idx]
    user = user_id[key]


    for item in rec_item_id :
        final.append((user, item))

    cnt+=1

    if cnt%5000 == 0 :
        print(f"{cnt}/31360 complete")



5000/31360 complete
10000/31360 complete
15000/31360 complete
20000/31360 complete
25000/31360 complete
30000/31360 complete


In [59]:
info = pd.DataFrame(final, columns=['user','item'])
info.to_csv("submission_B4R.csv",index=False)
print("Inference Done!")

Inference Done!
